# Creació del Senyal

* Aqui es proposen dos funcions per crear un senyal. La primera crea un senyal aleatori de mida lenght. La segona, en canvi, força que hi hagi 8 zeros seguits en el senyal.

In [111]:
import random
def sequencia(length):
    #Sequencia de bits
    seq = []
    for i in range(length):
        seq.append(random.randint(0,1))
    return seq

In [177]:
import random
def sequenciaWithScrambling(length):
    seq = []
    for i in range(length):
        seq.append(random.randint(0,1))
    indexToInsert = random.randint(0,length-8)
    for i in range(8):
        seq[indexToInsert+i] = 0
    return seq

# Codificació

Aqui estàn constituits els principals mètodes de codificació vists a classe. 
ATENCIO: 
* ** 0 significa absencia de senyal.**
* ** 1 significa pols positiu**
* **-1 significa pols negatiu**
* Per Manchester: **UpToDown significa "desde d'alt cap a baix" i DownToUp "de baix a dalt"


In [51]:
#Non Return to Zero - Level
def nrzLevel(seq):
    nrz = []
    for bit in seq:
        if bit ==1:
            nrz.append(-1) #-1 == pols negatiu
        else:
            nrz.append(0)
    return nrz

In [52]:
#Non Return to Zero inverted
def nrzInverted(seq):
    nrzi = []

    for i in range(len(seq)):
        if i == 0: #Inici de la sequencia
            if seq[i]==0:
                nrzi.append(0)
            else:
                nrzi.append(1)
        else:
            if seq[i] == 0:
                nrzi.append(nrzi[len(nrzi)-1]) #Zero == continuació
            else:
                if nrzi[i-1] == 1: #Si l'anterior tambe era un 1, canvi
                    nrzi.append(-1)
                else:
                    nrzi.append(1)
                    
    return nrzi

In [247]:
#Bipolar AMI. Té un segon parametre "pseudo". És un Boolea. Si esta activiat, passa a ser un metode de 
#codificacio pseudoternaria
def bipolarAMIpseudo(seq,pseudo):
    
    bipAmi = []
    level = 0 #Variable que em va controlant quin valor va prendre l'ultim 1
    zero = 0
    if pseudo:
        zero = 1
    
    for i in range(len(seq)):
        if seq[i] == zero:
            bipAmi.append(0)
        else:
            if level == 1:
                bipAmi.append(-1)
                level = 0
            else:
                bipAmi.append(1)
                level = 1
                
    return bipAmi

In [248]:
#Codificacio Manchester
def Manchester(seq):
    man = []
    for bit in seq:
        if bit == 1:
            man.append("DownToUp")
        else:
            man.append("UpToDown")
    return man

# Descodificació

* Aquí estàn implementats els mètodes de descodificació per cada mètode de codificació

In [249]:
#Non Return to Zero Level
def nrzLevelDecoder(seq):
    nrz = []

    for bit in seq:
        if bit ==-1:
            nrz.append(1) #-1 == pols negatiu
        else:
            nrz.append(0)

    return nrz

In [282]:
#Non Return to Zero Inverted
def nrzInvertedDecoder(seq):
    nrzi = []

    for i in range(len(seq)):
        if i == 0: #Inici de la sequencia
            if seq[i]==0:
                nrzi.append(0)
            else:
                nrzi.append(1)
        else:
            if seq[i] == seq[i-1]: #Si els dos ultims polsos son iguals, nomes pot ser un zero
                nrzi.append(0)
            else:
                nrzi.append(1)
    return nrzi

In [272]:
#Bipolar AMI i Pseudoternaria
def bipolarAmiPseudoDecoder(seq,pseudo):
    bipAmi = []
    zero = 0 if not pseudo else 1 

    for i in range(len(seq)):
        if seq[i] == zero: #Un zero nomes pot haver estat causat per un zero.
            bipAmi.append(0)
        else:
            if seq[i] == -1 and pseudo: #Si tinc un -1 i estic en pseudoternaria, l'ha causat un zero
                bipAmi.append(0)
            else:
                bipAmi.append(1) #Si no, un 1 nomes el pot causar un 1
    return bipAmi

In [273]:
#Manchester Decoding
def ManchesterDecoder(seq):
    man = []
    
    for bit in seq:
        if bit == "DownToUp":
            man.append(1)
        else:
            man.append(0)
    return man

# Scrambling


Aqui hi ha la gestió del scrambling.
* La primera funcio retorna l'index d'on comença la sequencia de 8 zeros en un senyal. Retorna -1 si no hi ha 8 zeros en el senyal. 

* b8zs transforma els 8 zeros segons el seu mètode
* b8zsDecoder retorna aquets 8 zeros al senyal

In [188]:
def vuitZeros(seq):
    vuit = [0,0,0,0,0,0,0,0]
    i, n, m = -1, len(seq), len(vuit)
    try:
        while True:
            i = seq.index(vuit[0], i + 1, n - m + 1)
            if vuit == seq[i:i + m]:
                return i
    except ValueError:
        return -1

In [189]:
def b8zs(seq,index):
    #00+-0-+
    modifiedSeq = seq
    modifiedSeq[index+2] = 1
    modifiedSeq[index+3] = -1
    modifiedSeq[index+5] = -1
    modifiedSeq[index+6] = 1
    return modifiedSeq
    

In [201]:
def b8zsDecoder(seq,index):
    #00+-0-+
    originalSeq = seq
    originalSeq[index+2] = 0
    originalSeq[index+3] = 0
    originalSeq[index+5] = 0
    originalSeq[index+6] = 0
    return originalSeq

# Correct Codification

In [285]:
#Funcio que comprova si dos senyals son iguals
def correctCodification(senyal,decodedSenyal):
    return senyal == decodedSenyal

# Main

A la funció main se l'hi ha de passar el codi de la codificació a codificar i la longitud del nostre senyal.

CODIS:
* 1: Non Return to Zero Level
* 2: NRZ Inverted
* 3: Bipolar AMI
* 4: Pseudoternaria
* 5: Manchester

El meu main comprova si en el senyal creat hi ha perill de sincronisme (hi ha 8 zeros seguits). Si exiseix aquest perill, aplicara b8zs. 

Després, retorna si la codificació ha estat correcte.

In [278]:
def main(code,length):
    
    codedSignal = []
    decodedSignal = []
    codificacio = ""
    
    senyal = sequencia(length)
    if vuitZeros(senyal) != -1:
        print("La senyal pot tenir pèrdua de sincronisme, utilitzarem el metode b8zs")
        codificacio = "B8ZS"
        codedSignal = b8zs(senyal,vuitZeros(senyal))
        decodedSignal = b8zsDecoder(senyal,vuitZeros(senyal))
        
    else:
        if code == 1:
            codificacio = "NRZ Level"
            codedSignal = nrzLevel(senyal)
            decodedSignal = nrzLevelDecoder(codedSignal)
        if code == 2:
            codificacio = "NRZ Inverted"
            codedSignal = nrzInverted(senyal)
            decodedSignal = nrzInvertedDecoder(codedSignal)
        if code == 3:
            codificacio = "Bipolar AMI"
            codedSignal = bipolarAMIpseudo(senyal,False)
            decodedSignal = bipolarAmiPseudoDecoder(codedSignal,False)
        if code == 4:
            codificacio = "Pseudoternaria"
            codedSignal = bipolarAMIpseudo(senyal,True)
            decodedSignal = bipolarAmiPseudoDecoder(codedSignal,True)
        if code == 5:
            codificacio = "Manchester"
            codedSignal = Manchester(senyal)
            decodedSignal = ManchesterDecoder(codedSignal)
        
    if (correctCodification(senyal,decodedSignal)):
        print("Your signal is: ",senyal," .")
        print("Your coded signal is: ",codedSignal, " with, ",codificacio)
        print("Your Decoded signal is: ",decodedSignal," equal to your original Signal. Codification done Right!")
    else:
        print("There was a problem during codification. Original and Decoded signal don't match")

A continuació es posen d'exemple alguns "mains" interesants. Si es vol sempre scrambling, s'hauria de fer servir la funció senyalWithScrambling. 
Si es vol "causar" scrambling indirectament, la llargada del senyal ha de ser molt llarga

In [287]:
#NRZ Level
main(1,10)

Your signal is:  [1, 0, 0, 1, 1, 1, 1, 1, 1, 1]  .
Your coded signal is:  [-1, 0, 0, -1, -1, -1, -1, -1, -1, -1]  with,  NRZ Level
Your Decoded signal is:  [1, 0, 0, 1, 1, 1, 1, 1, 1, 1]  equal to your original Signal. Codification done Right!


In [289]:
#NRZ Inverted
main(2,10)

Your signal is:  [0, 0, 1, 1, 0, 0, 0, 1, 1, 1]  .
Your coded signal is:  [0, 0, 1, -1, -1, -1, -1, 1, -1, 1]  with,  NRZ Inverted
Your Decoded signal is:  [0, 0, 1, 1, 0, 0, 0, 1, 1, 1]  equal to your original Signal. Codification done Right!


In [290]:
#Bipolar AMI
main(3,10)

Your signal is:  [1, 1, 0, 0, 1, 0, 1, 1, 1, 1]  .
Your coded signal is:  [1, -1, 0, 0, 1, 0, -1, 1, -1, 1]  with,  Bipolar AMI
Your Decoded signal is:  [1, 1, 0, 0, 1, 0, 1, 1, 1, 1]  equal to your original Signal. Codification done Right!


In [291]:
#Pseudo
main(4,10)

Your signal is:  [0, 0, 0, 0, 1, 1, 1, 1, 1, 0]  .
Your coded signal is:  [1, -1, 1, -1, 0, 0, 0, 0, 0, 1]  with,  Pseudoternaria
Your Decoded signal is:  [0, 0, 0, 0, 1, 1, 1, 1, 1, 0]  equal to your original Signal. Codification done Right!


In [292]:
#Manchester
main(5,10)

Your signal is:  [0, 0, 1, 0, 1, 1, 1, 0, 1, 1]  .
Your coded signal is:  ['UpToDown', 'UpToDown', 'DownToUp', 'UpToDown', 'DownToUp', 'DownToUp', 'DownToUp', 'UpToDown', 'DownToUp', 'DownToUp']  with,  Manchester
Your Decoded signal is:  [0, 0, 1, 0, 1, 1, 1, 0, 1, 1]  equal to your original Signal. Codification done Right!


In [294]:
#SCRAMBLING
main(3,200)

La senyal pot tenir pèrdua de sincronisme, utilitzarem el metode b8zs
Your signal is:  [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, -1, 0, -1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0]  .
Your coded signal is:  [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0